In [1]:
import datetime as dtm
import requests
import pandas as pd
import time
import os
import numpy as np
import scipy as sp
import pickle
from xgboost import XGBRegressor, XGBClassifier
from xgboost import plot_importance
import matplotlib.pyplot as plt
import src.data_retriever as intrinio
import src.pso as pso
import src.trading_bot as bot
import random
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
%matplotlib inline

In [2]:
random.seed(42)
datefrom = dtm.datetime(2010, 1, 1)
dateto = dtm.datetime(2019, 3, 1)

In [3]:
#DOW 30
# stock_names = ['AAPL', 'CSCO', 'WMT', 'MCD', 'GS', 'AXP', 'BA', 'KO', 'V', 'WMT', 'UNH']
stock_names = ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'XOM', 'GS',
               'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG',
               'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS']
# stock_names = ['AAPL']

In [4]:
stocks_filename = 'stocks_dow30_full.csv'
if os.path.isfile(stocks_filename):
    stocks = pd.read_csv(stocks_filename)
    stocks['date'] = pd.to_datetime(stocks['date'], format="%Y-%m-%d")
else:
    stocks = intrinio.get_stocks_data(stock_names, datefrom, dateto)
    stocks.to_csv(stocks_filename, index_label='id', columns=stocks.columns)

### Подготовка к тестированию

In [5]:
train_datefrom = datefrom
train_dateto = dtm.datetime(2017, 1, 1)

valid_datefrom = train_dateto
valid_dateto = dtm.datetime(2018, 1, 1)

test_datefrom = dtm.datetime(2018, 1, 1)
test_dateto = dtm.datetime(2019, 1, 1)

In [6]:
data = stocks.drop(0).reset_index(drop=True)
data['date'] = data['date'].map(lambda x: x.timestamp())
#data.drop('date', axis=1)
test_data = data[data['stock_name'] == 'PFE']
#data = data[data['stock_name'] != 'KO']
data = data.drop('stock_name', axis=1)
test_data = test_data.drop('stock_name', axis=1)
test_data = test_data[test_data['date'] > test_datefrom.timestamp()]

In [7]:
#to_normalize = ['volume_prev1', 'ebitda']
#data['date_norm'] = MinMaxScaler().fit_transform(data['date'].as_matrix().reshape(-1, 1))
#data['volume_prev1'] = MinMaxScaler().fit_transform(data['volume_prev1'].as_matrix().reshape(-1, 1))
#data['ebitda'] = MinMaxScaler().fit_transform(data['ebitda'].as_matrix().reshape(-1, 1))

#lbl_encoder=LabelEncoder()
#data['stock_name'] = lbl_encoder.fit_transform(data['stock_name'].as_matrix().reshape(-1, 1))
#test_data['stock_name'] = lbl_encoder.transform(test_data['stock_name'].as_matrix().reshape(-1, 1))
# test_data = data[data['stock_name'] == 0]
# data = data[data['stock_name'] != 0]

In [8]:
#lbl_encoder.classes_

In [9]:
#lbl_encoder.inverse_transform(np.array([1]))

In [10]:
train_data = data[data['date'] <= train_dateto.timestamp()]
X_train = train_data.drop(['date','close','adj_close', 'volume'],axis=1)
print(len(X_train))
Y_train_regr = train_data['close']
Y_train_classify = train_data['close'] > train_data['open']

49336


In [11]:
valid_data = data[data['date'] > valid_datefrom.timestamp()]
valid_data = valid_data[valid_data['date'] < valid_dateto.timestamp()]
# valid_data = valid_data.query('(stock_name == 0)')
X_valid = valid_data.drop(['date','close','adj_close', 'volume'],axis=1)
print(len(X_valid))
Y_valid_regr = valid_data['close']
Y_valid_classify = valid_data['close'] > valid_data['open']

7028


In [12]:
#test_data = data[data['date'] > test_datefrom.timestamp()]
# test_data = test_data[test_data['date'] < test_dateto.timestamp()]
# test_data = test_data[test_data['stock_name'] == lbl_encoder.transform(np.array(['AAPL']))[0]]
# test_data = test_data[test_data['stock_name'] == lbl_encoder.transform(np.array(['UNH']))[0]]
# test_data = test_data[test_data['stock_name'] == lbl_encoder.transform(np.array(['MMM']))[0]]
len(test_data)

251

In [13]:
ts = time.time()
class_params = {
  'colsample_bytree': 0.6617438163723828,
  'eta': 0.1447537144916002,
  'max_depth': 32,
  'min_child_weight': 173,
  'n_estimators': 2725,
  'subsample': 0.31509874620111583,
  'seed': 42}
model_class = XGBClassifier(**class_params)

model_class.fit(
        X_train, 
        Y_train_classify, 
        eval_set=[(X_train, Y_train_classify), (X_valid, Y_valid_classify)], 
        verbose=True,         
        eval_metric='auc',
        early_stopping_rounds = 10)
    
time.time() - ts

[0]	validation_0-auc:0.635805	validation_1-auc:0.6199
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.660028	validation_1-auc:0.642332
[2]	validation_0-auc:0.674551	validation_1-auc:0.656444
[3]	validation_0-auc:0.680007	validation_1-auc:0.663101
[4]	validation_0-auc:0.684594	validation_1-auc:0.667825
[5]	validation_0-auc:0.688123	validation_1-auc:0.671434
[6]	validation_0-auc:0.689657	validation_1-auc:0.674561
[7]	validation_0-auc:0.691287	validation_1-auc:0.677008
[8]	validation_0-auc:0.69426	validation_1-auc:0.680018
[9]	validation_0-auc:0.694084	validation_1-auc:0.680174
[10]	validation_0-auc:0.696305	validation_1-auc:0.680043
[11]	validation_0-auc:0.698887	validation_1-auc:0.682349
[12]	validation_0-auc:0.701227	validation_1-auc:0.684304
[13]	validation_0-auc:0.703799	validation_1-auc:0.687924
[14]	validation_0-auc:0.704949	validation_1-auc:0.687966
[

23.075300931930542

In [14]:
test_datefrom

datetime.datetime(2018, 1, 1, 0, 0)

In [15]:
test_dateto

datetime.datetime(2019, 1, 1, 0, 0)

In [19]:
def strategy(row):
    strategy_input = pd.DataFrame([row])
    strategy_input = strategy_input.drop(['date','close','adj_close', 'volume'], axis=1)
    decision = 'buy' if model_class.predict(strategy_input) else 'sell'
    return decision

cash_res = bot.simulate_trading(strategy, test_data, initial_cash=11000, multipl=100, interest_percent=0.05)
print(cash_res)

sell: open: 36.38, cash=10992.178,  diff=5.999999999999517, interest=1.822
sell: open: 36.45, cash=10964.3425,  diff=25.9999999999998, interest=1.8355000000000001
sell: open: 36.71, cash=10954.503,  diff=7.9999999999998295, interest=1.8395000000000001
sell: open: 36.82, cash=10948.66,  diff=3.9999999999999147, interest=1.843
sell: open: 36.72, cash=10973.8375,  diff=-26.999999999999602, interest=1.8225000000000002
sell: open: 36.44, cash=10975.017,  diff=-3.0000000000001137, interest=1.8204999999999998
sell: open: 36.34, cash=10960.1935,  diff=12.999999999999545, interest=1.8235000000000001
sell: open: 36.64, cash=10966.3655,  diff=-7.9999999999998295, interest=1.828
sell: open: 36.63, cash=10973.5385,  diff=-9.000000000000341, interest=1.827
buy: open: 36.57, cash=10974.710000000001, diff=3.0000000000001137, interest=1.8285
buy: open: 36.82, cash=11008.869, diff=35.99999999999994, interest=1.841
sell: open: 37.17, cash=11025.0195,  diff=-17.99999999999997, interest=1.8495000000000001


buy: open: 36.0, cash=11921.225249999996, diff=32.99999999999983, interest=1.8
buy: open: 36.39, cash=11915.405749999996, diff=-3.9999999999999147, interest=1.8195000000000001
buy: open: 36.54, cash=11946.578749999997, diff=32.99999999999983, interest=1.827
buy: open: 37.0, cash=11955.728749999997, diff=10.999999999999943, interest=1.85
buy: open: 37.27, cash=11942.865249999995, diff=-11.000000000000654, interest=1.8635000000000002
buy: open: 37.3, cash=11954.000249999996, diff=13.000000000000256, interest=1.8649999999999998
buy: open: 37.25, cash=11948.137749999996, diff=-3.9999999999999147, interest=1.8625
buy: open: 37.41, cash=11957.267249999997, diff=11.000000000000654, interest=1.8704999999999998
buy: open: 37.5, cash=11958.392249999997, diff=3.0000000000001137, interest=1.875
buy: open: 37.48, cash=11946.518249999997, diff=-9.999999999999432, interest=1.8739999999999999
buy: open: 37.46, cash=11963.645249999998, diff=18.999999999999773, interest=1.873
buy: open: 37.59, cash=1196